# Segmenting and Clustering Neighborhoods in Toronto - Varnan Goodwin

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [3]:
! pip install BeautifulSoup4

In [4]:
# importing libraries
from bs4 import BeautifulSoup
import urllib.request
import re

In [5]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'html.parser')

In [6]:
My_table = soup.find('table',{'class':'wikitable sortable'})


In [7]:
table_rows = My_table.find_all('tr')
Row_list =[] 
for tr in table_rows:    
    td = tr.find_all('td')
    row=   [tr.text.rstrip() for tr in td]  
    Row_list.append(row) 

Row_list.pop(0)
df = pd.DataFrame((Row_list), columns =['PostalCode','Borough','Neighborhood']) 

In [8]:
df.shape

(287, 3)

In [9]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

unassignedBorough = df[ df['Borough'] == 'Not assigned'].index
df.drop(unassignedBorough , inplace=True)

In [10]:
df[df['Borough'] == 'Not assigned'] 


,PostalCode,Borough,Neighborhood


In [11]:
df.shape

(210, 3)

In [12]:
df[df['PostalCode'] == 'M1C'] 


,PostalCode,Borough,Neighborhood
26,M1C,Scarborough,Highland Creek
27,M1C,Scarborough,Rouge Hill
28,M1C,Scarborough,Port Union


In [13]:
# More than one neighborhood exists in one postal code area. For example, in the table on the Wikipedia page, you will notice that M1C is listed three times ..
# These two rows will be combined into one row with the neighborhoods separated with a comma  -->

df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()


In [14]:
df.shape

(103, 3)

In [15]:
df[df['PostalCode'] == 'M1C'] 


,PostalCode,Borough,Neighborhood
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"


In [16]:
df[df['Neighborhood'] == 'Not assigned'] 

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


In [17]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
df.Neighborhood[df.Neighborhood == 'Not assigned'] = df.Borough


In [18]:
df[df['Neighborhood'] == 'Not assigned'] 

,PostalCode,Borough,Neighborhood


In [19]:
df[df['PostalCode'] == 'M9A'] 

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Queen's Park


In [ ]:
# Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In [20]:
df.shape

(103, 3)

## Get the latitude and the longitude coordinates of each neighborhood.

In [21]:
# download csv file
!wget --quiet http://cocl.us/Geospatial_data

print('CSV file downloaded!')

CSV file downloaded!


In [22]:
df_Geo = pd.read_csv('Geospatial_Coordinates.csv')

In [23]:
df_Geo.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 


In [24]:
df_Geo.shape

(103, 3)

#### Merge dataframes df and df_Geo

In [26]:
df_merged = pd.merge(left=df,right=df_Geo, left_on='PostalCode', right_on='PostalCode')

In [27]:
df_merged[df_merged['PostalCode'] == 'M9V'] 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [28]:
df_merged.shape

(103, 5)

# Explore and cluster the neighborhoods in Toronto.

In [29]:
df_Toronto = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [30]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Toronto['Borough'].unique()),
        df_Toronto.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City.

In [31]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [104]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

##### For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Cen. So let's slice the original dataframe and create a new dataframe of the DownTown Toronto.

In [33]:
DowntownToronto_data = df_Toronto[df_Toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DowntownToronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


In [34]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6563221, -79.3809161.


##### As we did with all of Toronco City, let's visualizat Downtowin and the neighborhoods in it.

In [103]:
# create map of Manhattan using latitude and longitude values
map_DowntownToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DowntownToronto_data['Latitude'], DowntownToronto_data['Longitude'], DowntownToronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DowntownToronto)  
    
map_DowntownToronto

##### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

##### Define Foursquare Credentials and Version

In [36]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

##### Let's explore the first neighborhood in our dataframe.

In [37]:
DowntownToronto_data.loc[0, 'Neighborhood']

'Rosedale'

Get the neighborhood's latitude and longitude values.

In [38]:
neighborhood_latitude = DowntownToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DowntownToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DowntownToronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


##### Now, let's get the top 100 venues that are in Rosedale within a radius of 500 meters.

In [39]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)



In [41]:
results = requests.get(url).json()


In [42]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


#### Explore Neighborhoods in Downtown Toronto

In [45]:
# a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
# code to run the above function on each neighborhood and create a new dataframe called *Downtown Toronto_venues*.


DowntownToronto_venues = getNearbyVenues(names=DowntownToronto_data['Neighborhood'],
                                   latitudes=DowntownToronto_data['Latitude'],
                                   longitudes=DowntownToronto_data['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Queen's Park


In [48]:
print(DowntownToronto_venues.shape)
DowntownToronto_venues.head()

(1314, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown,St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [49]:
# Venues were returned for each neighborhood
DowntownToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",48,48,48,48,48,48
Central Bay Street,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17
Church and Wellesley,84,84,84,84,84,84
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [51]:
# unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(DowntownToronto_venues['Venue Category'].unique())))

There are 211 uniques categories.


#### Analyze Each Neighborhood

In [52]:
# one hot encoding
DowntownToronto_onehot = pd.get_dummies(DowntownToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DowntownToronto_onehot['Neighborhood'] = DowntownToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DowntownToronto_onehot.columns[-1]] + list(DowntownToronto_onehot.columns[:-1])
DowntownToronto_onehot = DowntownToronto_onehot[fixed_columns]

DowntownToronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [54]:
DowntownToronto_onehot.shape

(1314, 211)

##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [58]:
DowntownToronto_grouped = DowntownToronto_onehot.groupby('Neighborhood').mean().reset_index()


In [60]:
DowntownToronto_grouped.shape

(19, 211)

 ##### Print each neighborhood along with the top 2 most common venues

In [63]:
num_top_venues = 2

for hood in DowntownToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DowntownToronto_grouped[DowntownToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
             venue  freq
0  Airport Service  0.19
1   Airport Lounge  0.12


----Cabbagetown,St. James Town----
         venue  freq
0       Bakery  0.06
1  Coffee Shop  0.06


----Central Bay Street----
         venue  freq
0  Coffee Shop  0.16
1         Café  0.05


----Chinatown,Grange Park,Kensington Market----
                   venue  freq
0                   Café  0.06
1  Vietnamese Restaurant  0.05


----Christie----
           venue  freq
0           Café  0.18
1  Grocery Store  0.18


----Church and Wellesley----
              venue  freq
0       Coffee Shop  0.08
1  Sushi Restaurant  0.05


----Commerce Court,Victoria Hotel----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.

##### put that into a *pandas* dataframe

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
DowntownToronto_venues_sorted = pd.DataFrame(columns=columns)
DowntownToronto_venues_sorted['Neighborhood'] = DowntownToronto_grouped['Neighborhood']

for ind in np.arange(DowntownToronto_grouped.shape[0]):
    DowntownToronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DowntownToronto_grouped.iloc[ind, :], num_top_venues)

DowntownToronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Sushi Restaurant,Bakery,Burger Joint,Restaurant,Hotel,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Cheese Shop,Café,Japanese Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Plane,Boutique,Rental Car Location,Bar,Sculpture Garden,Harbor / Marina
3,"Cabbagetown,St. James Town",Coffee Shop,Bakery,Flower Shop,Italian Restaurant,Market,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Japanese Restaurant,Ice Cream Shop,Bubble Tea Shop,Bar,Bakery


##### Cluster Neighborhoods

##### Run *k*-means to cluster the neighborhood into 5 clusters.

In [94]:
# set number of clusters
kclusters = 5

DowntownToronto_grouped_clustering = DowntownToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DowntownToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [95]:
DowntownToronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Coffee Shop,Bakery,Flower Shop,Italian Restaurant,Market,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Theater,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Gym
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Café,Brewery,Event Space,Restaurant
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store,Bubble Tea Shop,Bookstore


##### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [96]:


DowntownToronto_merged = DowntownToronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DowntownToronto_merged = DowntownToronto_merged.join(DowntownToronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DowntownToronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Coffee Shop,Bakery,Flower Shop,Italian Restaurant,Market,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Theater,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Gym
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Café,Brewery,Event Space,Restaurant
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store,Bubble Tea Shop,Bookstore


In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DowntownToronto_merged['Latitude'], DowntownToronto_merged['Longitude'], DowntownToronto_merged['Neighborhood'], DowntownToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### We can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

In [98]:
# Cluster 1
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 0, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Bakery,Flower Shop,Italian Restaurant,Market,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant
2,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Theater,Fast Food Restaurant,Burger Joint,Bubble Tea Shop,Gym
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store,Bubble Tea Shop,Bookstore
5,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Cosmetics Shop,Clothing Store,Breakfast Spot,Bakery,Cocktail Bar,Hotel,Beer Bar
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Cheese Shop,Café,Japanese Restaurant
7,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Japanese Restaurant,Ice Cream Shop,Bubble Tea Shop,Bar,Bakery
8,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Sushi Restaurant,Bakery,Burger Joint,Restaurant,Hotel,Thai Restaurant
9,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Fried Chicken Joint,History Museum
10,Downtown Toronto,0,Coffee Shop,Hotel,Café,Bar,Restaurant,Seafood Restaurant,Italian Restaurant,Gastropub,Steakhouse,American Restaurant
11,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Gym,Gastropub,American Restaurant,Steakhouse,Italian Restaurant


In [99]:
# Cluster 2
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 1, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [100]:
# Cluster 3
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 2, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Plane,Boutique,Rental Car Location,Bar,Sculpture Garden,Harbor / Marina


In [101]:
# Cluster 4
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 3, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,3,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Café,Brewery,Event Space,Restaurant
18,Downtown Toronto,3,Coffee Shop,Park,Gym,College Auditorium,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café,Portuguese Restaurant


In [102]:
# Cluster 5
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 4, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,4,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Restaurant,Baby Store,Candy Store,Convenience Store
